In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import numpy as np
import seaborn as sns
from pathlib import Path
sns.set_style('whitegrid')  # Set whitegrid style from Seaborn
#import mplfinance as mpf
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook"
import plotly.express as px
import matplotlib.ticker as tkr
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None
from collections import defaultdict
from scipy import stats
from datetime import datetime
import pytz
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 500)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import neptune as neptune
from scipy.interpolate import CubicSpline


In [20]:
#merged_df = pd.read_csv('C:/Users/kathe/Documents/ppc-isp-energy-offers/data/PHASE3/demo.csv')
merged_df = pd.read_csv('demo.csv')
merged_df.drop(columns = ['Unnamed: 0'],axis=1,inplace = True)

In [21]:
merged_df.head()

,datetime_from,price,price1,price2,price3
0,2023-07-28 00:00:00,107.99,98.83,95.05,107.123
1,2023-07-28 01:00:00,106.04,95.19,93.70,104.801
2,2023-07-28 02:00:00,106.10,97.00,87.98,102.977
3,2023-07-28 03:00:00,105.00,100.36,89.45,109.143
4,2023-07-28 04:00:00,102.22,109.23,101.46,116.560


In [22]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8256 entries, 0 to 8255
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   datetime_from  8256 non-null   object 
 1   price          8256 non-null   float64
 2   price1         8256 non-null   float64
 3   price2         8256 non-null   float64
 4   price3         8256 non-null   float64
dtypes: float64(4), object(1)
memory usage: 322.6+ KB


In [23]:
df=merged_df.copy()

In [24]:
df['datetime_from'] = pd.to_datetime(df['datetime_from'])

In [25]:
# Adding hour, dayofweek, and month columns
df['hour'] = df['datetime_from'].dt.hour
df['dayofweek'] = df['datetime_from'].dt.dayofweek
df['month'] = df['datetime_from'].dt.month

# Flagging weekends
df['is_weekend'] = df['dayofweek'].apply(lambda x: 1 if x >= 5 else 0) 

In [26]:
df.head()

,datetime_from,price,price1,price2,price3,hour,dayofweek,month,is_weekend
0,2023-07-28 00:00:00,107.99,98.83,95.05,107.123,0,4,7,0
1,2023-07-28 01:00:00,106.04,95.19,93.70,104.801,1,4,7,0
2,2023-07-28 02:00:00,106.10,97.00,87.98,102.977,2,4,7,0
3,2023-07-28 03:00:00,105.00,100.36,89.45,109.143,3,4,7,0
4,2023-07-28 04:00:00,102.22,109.23,101.46,116.560,4,4,7,0


In [27]:
df['is_weekend'].value_counts()

0    5904
1    2352
Name: is_weekend, dtype: int64

In [28]:
from holidays import holidays

In [29]:
# Converting holiday dates to datetime
holiday_dates = [datetime.strptime(date, "%Y %B %d").date() for date, name in holidays]

In [30]:
# Adding holiday flag column
df['is_holiday'] = df['datetime_from'].dt.date.apply(lambda x: 1 if x in holiday_dates else 0)

# Display the enhanced dataframe
df.head()

,datetime_from,price,price1,price2,price3,hour,dayofweek,month,is_weekend,is_holiday
0,2023-07-28 00:00:00,107.99,98.83,95.05,107.123,0,4,7,0,0
1,2023-07-28 01:00:00,106.04,95.19,93.70,104.801,1,4,7,0,0
2,2023-07-28 02:00:00,106.10,97.00,87.98,102.977,2,4,7,0,0
3,2023-07-28 03:00:00,105.00,100.36,89.45,109.143,3,4,7,0,0
4,2023-07-28 04:00:00,102.22,109.23,101.46,116.560,4,4,7,0,0


In [31]:
# Assuming df_sin is your main dataframe, similar adjustments for other dataframes like data_val and data_test
# Encoding for day of the week
# Encoding for day of the week
df['dayofweek_sin'] = np.sin((df['dayofweek'] - df['dayofweek'].min()) * (2. * np.pi / 7))
df['dayofweek_cos'] = np.cos((df['dayofweek'] - df['dayofweek'].min()) * (2. * np.pi / 7))

# Encoding for month
df['month_sin'] = np.sin((df['month'] - 1) * (2. * np.pi / 12))
df['month_cos'] = np.cos((df['month'] - 1) * (2. * np.pi / 12))

# Encoding for hour
df['hour_sin'] = np.sin((df['hour'] - df['hour'].min()) * (2. * np.pi / 24))
df['hour_cos'] = np.cos((df['hour'] - df['hour'].min()) * (2. * np.pi / 24))
#

In [33]:
df.drop(columns = ['hour','dayofweek','month'], axis =1,inplace=True)

In [19]:
#df.to_csv('C:/Users/kathe/Documents/ppc-isp-energy-offers/data/PHASE3/allfeatures_sin.csv')
df.to_csv('allfeatures_sin.csv')

In [53]:
df.head()

,price,price1,price2,price3,is_weekend,is_holiday,dayofweek_sin,dayofweek_cos,month_sin,month_cos,hour_sin,hour_cos
datetime_from,,,,,,,,,,,,
2023-07-28 00:00:00,107.99,98.83,95.05,107.123,0,0,-0.433884,-0.900969,1.224647e-16,-1.0,0.000000,1.000000
2023-07-28 01:00:00,106.04,95.19,93.70,104.801,0,0,-0.433884,-0.900969,1.224647e-16,-1.0,0.258819,0.965926
2023-07-28 02:00:00,106.10,97.00,87.98,102.977,0,0,-0.433884,-0.900969,1.224647e-16,-1.0,0.500000,0.866025
2023-07-28 03:00:00,105.00,100.36,89.45,109.143,0,0,-0.433884,-0.900969,1.224647e-16,-1.0,0.707107,0.707107
2023-07-28 04:00:00,102.22,109.23,101.46,116.560,0,0,-0.433884,-0.900969,1.224647e-16,-1.0,0.866025,0.500000


### Select Specific Dates / Hours from a file with timestamps

In [43]:
df = df.set_index('datetime_from')

In [57]:
#Keep this as the 48 LAGS in the PAST to put into the model to find the next hour. LAST SEQUENCE of DATA
df_48 = df.loc['2024-07-01 00:00:00':'2024-07-02 23:00:00']
#Keep the last day to assume we PREDICT test our predictions
df_last_day = df.loc['2024-07-03 00:00:00':'2024-07-03 23:00:00']

In [58]:
df_last_day.reset_index(inplace=True)
df_48.reset_index(inplace=True)

In [59]:
df_48.to_csv('PAST_48HOURS_SIN.csv')
df_last_day.to_csv('NEXTDAY_FORECASTERS_SIN.csv')